<h1 align="center"> Исследование: Mongo vs Clickhouse </h1>

In [1]:
from faker import Faker
import uuid

fake = Faker()

In [2]:
# Количество записей и размер пачки
total_records = 1_000_000
batch_size = 100_000

In [3]:
# Предварительно сгенерированные UUID для пользователей и фильмов
user_ids = [str(uuid.uuid4()) for _ in range(5_000)]
movie_ids = [str(uuid.uuid4()) for _ in range(10_000)]

## Вставка в Mongo

In [4]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['online_cinema']


# Функция для вставки лайков
def insert_likes():
    for _ in range(total_records // batch_size):
        batch_data = [{
            'movie_id': fake.random_element(movie_ids),
            'user_id': fake.random_element(user_ids),
            'rating': fake.random_int(min=0, max=10)
        } for _ in range(batch_size)]
        db.likes.insert_many(batch_data)
    print("Лайки и дизлайки вставлены.")


# Функция для вставки рецензий
def insert_reviews():
    for _ in range(total_records // batch_size):
        batch_data = [{
            'movie_id': fake.random_element(movie_ids),
            'user_id': fake.random_element(user_ids),
            'text': fake.text(max_nb_chars=200),
            'date_published': fake.date_time_between(start_date='-2y', end_date='now'),
            'likes': fake.random_int(min=0, max=1000),
            'dislikes': fake.random_int(min=0, max=1000),
            'user_rating': fake.random_int(min=0, max=10)
        } for _ in range(batch_size)]
        db.reviews.insert_many(batch_data)
    print("Рецензии вставлены.")


# Функция для вставки закладок
def insert_bookmarks():
    for _ in range(total_records // batch_size):
        batch_data = [{
            'user_id': fake.random_element(user_ids),
            'movie_id': fake.random_element(movie_ids),
            'added_date': fake.date_time_this_year(before_now=True, after_now=False)
        } for _ in range(batch_size)]
        db.bookmarks.insert_many(batch_data)
    print("Закладки вставлены.")

In [5]:
%%time
insert_likes()
insert_reviews()
insert_bookmarks()

Лайки и дизлайки вставлены.
Рецензии вставлены.
Закладки вставлены.
CPU times: user 2min 34s, sys: 16 s, total: 2min 50s
Wall time: 3min 6s


## Тестирование Mongo

In [6]:
%%time
movie_id_example = fake.random_element(movie_ids)
likes_count = db.likes.count_documents({'movie_id': movie_id_example, 'rating': {'$gt': 5}})
dislikes_count = db.likes.count_documents({'movie_id': movie_id_example, 'rating': {'$lte': 5}})

print(f"Количество лайков у фильма {movie_id_example}: {likes_count}")
print(f"Количество дизлайков у фильма {movie_id_example}: {dislikes_count}")

Количество лайков у фильма f7437cde-99e4-4b46-9ead-863616918069: 46
Количество дизлайков у фильма f7437cde-99e4-4b46-9ead-863616918069: 52
CPU times: user 1.35 ms, sys: 945 µs, total: 2.3 ms
Wall time: 507 ms


In [7]:
%%time
movie_id_example = fake.random_element(movie_ids)
avg_rating = db.likes.aggregate([
    {'$group': {
        '_id': '$movie_id',
        'averageRating': {'$avg': '$rating'}
    }},
    {'$match': {'_id': movie_id_example}}
])

for doc in avg_rating:
    print(f"Средняя оценка фильма {movie_id_example}: {doc['averageRating']}")

Средняя оценка фильма 93af0be7-5b49-48d8-873d-3a77c0aeda56: 4.681818181818182
CPU times: user 1.57 ms, sys: 1.54 ms, total: 3.11 ms
Wall time: 161 ms


In [8]:
%%time
user_id_example = fake.random_element(user_ids)

liked_movies = db.likes.find({
    'user_id': user_id_example,
    'rating': {'$gte': 6}  # Оценка 6 и выше считается лайком
}).sort('rating', -1)

print(f"Фильмы, понравившиеся пользователю {user_id_example}, отсортированные по убыванию рейтинга:")
for movie in liked_movies:
    print(f"Movie ID: {movie['movie_id']} - Rating: {movie['rating']}")

Фильмы, понравившиеся пользователю b598b803-1b59-4ea6-961d-4be4137eff69, отсортированные по убыванию рейтинга:
Movie ID: 9d55e0b4-b8ca-4c4b-924b-c20c5b6b1254 - Rating: 10
Movie ID: b66958e7-c2c3-48d1-9e85-d4a62d29c31b - Rating: 10
Movie ID: 6c1bc485-4904-4420-92a1-c8ce7042fb88 - Rating: 10
Movie ID: 26cd2364-abe3-40f2-8217-5b5eae00ad22 - Rating: 10
Movie ID: 8838f0c4-394e-4a6b-bd47-21a14004f598 - Rating: 10
Movie ID: 32a0ef35-cdd9-4c29-8c94-b8d773bf739b - Rating: 10
Movie ID: 9b32130e-32ba-4de1-b438-cf9ad8409347 - Rating: 10
Movie ID: 1939827d-94ca-45fe-9726-e213271180ea - Rating: 10
Movie ID: ce5452ca-97f0-48e8-954a-db69866c33bf - Rating: 10
Movie ID: 6dc90bb9-98b3-48d9-8846-0644df46da68 - Rating: 10
Movie ID: 7e8c39a1-83b1-4b69-abff-877a2ea928f0 - Rating: 10
Movie ID: 98ba4715-12cb-4a31-bce8-f6256e463ff7 - Rating: 10
Movie ID: 8891384a-cbc9-4a02-a9b6-2f5bc8d916e4 - Rating: 10
Movie ID: bd5b7cc0-c94b-43ec-8035-dbd727ce5317 - Rating: 10
Movie ID: 83f9a755-fb08-4375-8f03-0f1e5953204e - 

In [9]:
from clickhouse_driver import connect

clickhouse = connect(dsn='clickhouse://localhost:9000/default?user=default').cursor()

# Создание таблиц
clickhouse.execute('''
CREATE TABLE IF NOT EXISTS likes (
    movie_id UUID,
    user_id UUID,
    rating Int8
) ENGINE = MergeTree()
ORDER BY (movie_id, user_id);
''')

clickhouse.execute('''
CREATE TABLE IF NOT EXISTS reviews (
    movie_id UUID,
    user_id UUID,
    text String,
    date_published DateTime,
    likes Int32,
    dislikes Int32,
    user_rating Int8
) ENGINE = MergeTree()
ORDER BY (movie_id, user_id);
''')

clickhouse.execute('''
CREATE TABLE IF NOT EXISTS bookmarks (
    user_id UUID,
    movie_id UUID,
    added_date DateTime
) ENGINE = MergeTree()
ORDER BY (user_id, movie_id);
''')

## Вставка в ClickHouse

In [10]:
# Функция для вставки лайков
def insert_likes():
    for _ in range(total_records // batch_size):
        batch_data = [(
            fake.random_element(user_ids),
            fake.random_element(movie_ids),
            fake.random_int(min=0, max=10)
        ) for _ in range(batch_size)]
        clickhouse.executemany('INSERT INTO likes (user_id, movie_id, rating) VALUES', batch_data)
    print("Лайки и дизлайки вставлены.")


# Функция для вставки рецензий
def insert_reviews():
    for _ in range(total_records // batch_size):
        batch_data = [(
            fake.random_element(user_ids),
            fake.random_element(movie_ids),
            fake.text(max_nb_chars=200),
            fake.date_time_between(start_date='-2y', end_date='now'),
            fake.random_int(min=0, max=1000),
            fake.random_int(min=0, max=1000),
            fake.random_int(min=0, max=10)
        ) for _ in range(batch_size)]
        clickhouse.executemany(
            'INSERT INTO reviews (user_id, movie_id, text, date_published, likes, dislikes, user_rating) VALUES',
            batch_data)
    print("Рецензии вставлены.")


# Функция для вставки закладок
def insert_bookmarks():
    for _ in range(total_records // batch_size):
        batch_data = [(
            fake.random_element(user_ids),
            fake.random_element(movie_ids),
            fake.date_time_this_year(before_now=True, after_now=False)
        ) for _ in range(batch_size)]
        clickhouse.executemany('INSERT INTO bookmarks (user_id, movie_id, added_date) VALUES', batch_data)
    print("Закладки вставлены.")

In [11]:
%%time
insert_likes()
insert_reviews()
insert_bookmarks()

Лайки и дизлайки вставлены.
Рецензии вставлены.
Закладки вставлены.
CPU times: user 2min 35s, sys: 15.3 s, total: 2min 50s
Wall time: 2min 53s


## Тестирование ClickHouse

In [12]:
%%time
movie_id_example = fake.random_element(movie_ids)

# Запрос на подсчёт количества лайков и дизлайков для конкретного фильма
query = f"""
SELECT 
    countIf(rating > 5) as likes, 
    countIf(rating <= 5) as dislikes 
FROM likes 
WHERE movie_id = '{movie_id_example}'
"""

clickhouse.execute(query)

result = clickhouse.fetchone()
likes_count, dislikes_count = result

print(f"Количество лайков у фильма {movie_id_example}: {likes_count}")
print(f"Количество дизлайков у фильма {movie_id_example}: {dislikes_count}")

Количество лайков у фильма c949a2ca-d00c-4da6-8352-a7edb797d6f3: 38
Количество дизлайков у фильма c949a2ca-d00c-4da6-8352-a7edb797d6f3: 58
CPU times: user 1.05 ms, sys: 1.02 ms, total: 2.07 ms
Wall time: 47.5 ms


In [13]:
%%time
movie_id_example = fake.random_element(movie_ids)

# Запрос на получение средней оценки для конкретного фильма
query_avg_rating = f"""
SELECT AVG(rating) AS averageRating
FROM likes
WHERE movie_id = '{movie_id_example}'
"""

clickhouse.execute(query_avg_rating)

avg_rating = clickhouse.fetchone()
print(f"Средняя оценка фильма {movie_id_example}: {avg_rating[0]}")

Средняя оценка фильма e7f127a4-b538-41f9-bfb6-f496a9edd565: 4.602150537634409
CPU times: user 1.06 ms, sys: 858 µs, total: 1.92 ms
Wall time: 7.24 ms


In [14]:
%%time
user_id_example = fake.random_element(user_ids)

# Запрос на получение списка фильмов, понравившихся пользователю, с рейтингом выше 5
query_liked_movies = f"""
SELECT movie_id, rating
FROM likes
WHERE user_id = '{user_id_example}' AND rating >= 6 
ORDER BY rating DESC
"""

clickhouse.execute(query_liked_movies)

liked_movies = clickhouse.fetchall()

print(f"Фильмы, понравившиеся пользователю {user_id_example}, отсортированные по убыванию рейтинга:")
for movie in liked_movies:
    print(f"Movie ID: {movie[0]} - Rating: {movie[1]}")

Фильмы, понравившиеся пользователю 6e7cd7ca-5351-4420-bb96-2555d391ac13, отсортированные по убыванию рейтинга:
Movie ID: 25182b67-1eea-4dc3-8112-7a76a4fb857d - Rating: 10
Movie ID: 3bfb4f78-1533-4279-8d2b-caa8cec6ca56 - Rating: 10
Movie ID: fc287bb1-52c5-41f7-92a7-284c122bf326 - Rating: 10
Movie ID: bfb39ff8-eb2b-4980-ad0f-d0786a5ac9c5 - Rating: 10
Movie ID: 5c63c3ee-a2f2-4b3c-80f1-6d20fe073a2b - Rating: 10
Movie ID: 63c3ce24-d05a-472d-86d9-1a2876d77c52 - Rating: 10
Movie ID: 25c7090f-596c-4b69-8c26-04f62e324a19 - Rating: 10
Movie ID: 723d5a7a-3eef-471f-ad85-e961a81a2580 - Rating: 10
Movie ID: 30347eee-f878-4985-9faa-3774733cd685 - Rating: 10
Movie ID: be802654-b101-4875-bc93-eb0d93f3ea09 - Rating: 10
Movie ID: 01cc0e1c-2ca8-4e72-89a5-177bccfbac0b - Rating: 10
Movie ID: 4de96e40-2faa-456c-8a7c-a2f55004b1b5 - Rating: 10
Movie ID: 76989bc2-2a3d-4318-92e5-931bea5049c4 - Rating: 10
Movie ID: dbe71c73-a7ee-40bc-92ef-a6ca3b5e7adb - Rating: 10
Movie ID: b5f11e6d-8448-4c23-a47e-89584078a013 - 